In [1]:
import numpy as np
import zarr
from PIL import Image
from IPython.display import display
from tqdm.notebook import tqdm, trange

In [2]:
# Retina plane
z_r = -17.0
size_r = 17.0
n_r = 512
pitch_r = size_r / n_r

focus_distances = np.array([250.0, np.inf])
n_f = len(focus_distances)

# Pupil plane
n_p = 2**5
size_p = 5.0
pitch_p = size_p / n_p
z_p = 0.0

# View Plane
n_v = 45 * 2**5
pitch_v = pitch_p
size_v = n_v * pitch_v
z_v = size_v - size_p

# Retina estimate
lf_pitch_r = pitch_v * (-z_r/ z_v)
lf_n_r = int(size_r / lf_pitch_r)

print('[Sampled Light Field]')
print('Pupil Plane:     {:4} / {:5.2f} = {:5} mm       z_p = {:5} mm'.format(n_p,       1/pitch_p, size_p, z_p))
print('View Plane:      {:4} / {:5.2f} = {:5} mm       z_v = {:5} mm'.format(n_v,       1/pitch_v, size_v, z_v))
print('Retina Estimate: {:4} / {:5.2f} = {:5} mm    z_near = {:5} mm'.format(lf_n_r, 1/lf_pitch_r, size_r, z_v/2))

ij_p = np.stack(np.broadcast_arrays(np.arange(n_p).reshape(n_p, 1), np.arange(n_p).reshape(1, n_p)), axis=-1).reshape(n_p**2, 2)
xyz_p = np.concatenate(((ij_p - (n_p - 1)/2) * pitch_p, -np.ones((n_p**2, 1)) * z_p), axis=-1)

ij_v = np.stack(np.broadcast_arrays(np.arange(n_v).reshape(n_v, 1), np.arange(n_v).reshape(1, n_v)), axis=-1).reshape(n_v**2, 2)
xyz_v = np.concatenate(((ij_v - (n_v - 1)/2) * pitch_v, -np.ones((n_v**2, 1)) * z_v), axis=-1)

print('\nNumber of rays:', f'{len(ij_p):,}', '×', f'{len(ij_v):,}', '=', f'{len(ij_p) * len(ij_v):,}')

[Sampled Light Field]
Pupil Plane:       32 /  6.40 =   5.0 mm       z_p =   0.0 mm
View Plane:      1440 /  6.40 = 225.0 mm       z_v = 220.0 mm
Retina Estimate: 1408 / 82.82 =  17.0 mm    z_near = 110.0 mm

Number of rays: 1,024 × 2,073,600 = 2,123,366,400


In [3]:
# Plane A
z_a = 10.0
size_a = size_r * z_a / abs(z_r) + size_p
n_a = 3 * 2**5
pitch_a = size_a / n_a

# Plane B
z_b = 30.0
size_b = size_r * z_b / abs(z_r) + size_p
n_b = 7 * 2**5
pitch_b = size_b / n_b

# Retina estimate
ab_pitch_r = pitch_a * z_r / (z_a - z_b)
ab_n_r = int(size_r / ab_pitch_r)

print('[Display Light Field]')
print('Plane A:         {:3} / {:5.2f} = {:5} mm       z_a = {:5} mm'.format(n_a,       1/pitch_a, size_a, z_a))
print('Plane B:         {:3} / {:5.2f} = {:5} mm       z_b = {:5} mm'.format(n_b,       1/pitch_b, size_b, z_b))
print('Retina Estimate: {:3} / {:5.2f} = {:5} mm    z_near = {:5} mm'.format(ab_n_r, 1/ab_pitch_r, size_r, z_b/2))

ij_a = np.stack(np.broadcast_arrays(np.arange(n_a).reshape(n_a, 1), np.arange(n_a).reshape(1, n_a)), axis=-1).reshape(n_a**2, 2)
xyz_a = np.concatenate(((ij_a - (n_a - 1)/2) * pitch_a, -np.ones((n_a**2, 1)) * z_a), axis=-1)

ij_b = np.stack(np.broadcast_arrays(np.arange(n_b).reshape(n_b, 1), np.arange(n_b).reshape(1, n_b)), axis=-1).reshape(n_b**2, 2)
xyz_b = np.concatenate(((ij_b - (n_b - 1)/2) * pitch_b, -np.ones((n_b**2, 1)) * z_b), axis=-1)

print('\nNumber of elements:', f'{len(ij_a):,}', '×', f'{len(ij_b):,}', '=', f'{len(ij_a) * len(ij_b):,}')

[Display Light Field]
Plane A:          96 /  6.40 =  15.0 mm       z_a =  10.0 mm
Plane B:         224 /  6.40 =  35.0 mm       z_b =  30.0 mm
Retina Estimate: 128 /  7.53 =  17.0 mm    z_near =  15.0 mm

Number of elements: 9,216 × 50,176 = 462,422,016


In [ ]:
%%time
print('Processing rays...')

retina_colors = np.zeros((n_f, 3, n_r, n_r), dtype=np.float32)
retina_counts = np.zeros((n_f, 1, n_r, n_r), dtype=np.int32)

light_field = zarr.open('data/sponza.zarr', mode='r')

for id_p in trange(len(ij_p)):
    # Load ray values.
    lf_view = light_field[:, ij_p[id_p, 0], :, ij_p[id_p, 1], :].reshape(3, n_v**2)

    for id_f in range(n_f):
        z_f = focus_distances[id_f]
        # Retina intersection.
        xy_r = (xyz_p[id_p, 0:2] * (1 - z_v/z_f) - xyz_v[:, 0:2]) * (-z_r) / z_v
        ij_r = np.clip(np.floor_divide(xy_r, pitch_r) + n_r/2, 0, n_r - 1).astype(np.uint32)
        # For each ray...
        for id_v in range(len(ij_r)):
            retina_colors[id_f, :, ij_r[id_v, 0], ij_r[id_v, 1]] += lf_view[:, id_v]
            retina_counts[id_f, :, ij_r[id_v, 0], ij_r[id_v, 1]] += 1